In [1]:
from jsonrpc.proxy import JSONRPCProxy
daemon = JSONRPCProxy.from_url("http://localhost:5279/lbryapi")

In [2]:
daemon.get_balance() #this is just a test

4.06948306

In [3]:
import json
import requests
import datetime

In [4]:
#jacks reward function
COIN = 10**8
block_time = 150
blocks_per_ramp = 100
start_ramp_height = 5001
limit = 1083221000 * COIN
subsidy_interval = 1 << 5
max_reductions = 500
peak_reward = 500 * COIN
premine = 400000000 * COIN


def reward(n):
    assert n >= 0
    if n < 5001:
        return COIN
    elif n < 55001:
        r = 1 + ((n - ((n - start_ramp_height) % blocks_per_ramp) - start_ramp_height) / blocks_per_ramp)
        return r * COIN
    else:
        level = int((n - 55001) / subsidy_interval)
        reduction = int((-1 + int((8 * level) + 1) ** 0.5) / 2)
        while level >= ((reduction * reduction + reduction) >> 1):
            if ((reduction * reduction + reduction) >> 1) > level:
                reduction -= 1
            else:
                reduction += 1
        if reduction * COIN >= peak_reward:
            return 0
        return peak_reward - (reduction * COIN)

In [5]:
def block_velocity(block):
    list_of_tx = [daemon.get_transaction({'txid':txh}) for txh in block['tx']]
    #print list_of_tx
    list_of_tx_outs = [sum([int(out['value'])for out in tx['outputs']]) for tx in list_of_tx]
    return float(sum(list_of_tx_outs))/COIN

In [8]:
conv = COIN #deweys to 1 LBC
block_data = {}

#genesis
genesis_hash = "9c89283ba0f3227f6c03b70216b9f665f0118d5e0fa729cedf4fb34d6a34f463"

current_block=daemon.get_block({'blockhash': genesis_hash})

block_data[0] = {"height":0, 
                     "timestamp":datetime.datetime.fromtimestamp(int(current_block['time'])), 
                     "reward":reward(0)/COIN, 
                     "tx_count": len(current_block['tx']),
                     "tx_outs": block_velocity(current_block),
                     "difficulty": current_block['difficulty'],
                     "confirmations": current_block['confirmations']
                 }

In [9]:
block_data

{0: {'confirmations': 37281,
  'difficulty': 1.52587890625e-05,
  'height': 0,
  'reward': 1,
  'timestamp': datetime.datetime(2015, 10, 28, 14, 51, 31),
  'tx_count': 1,
  'tx_outs': 400000000.0}}

In [10]:
bhash=daemon.get_best_blockhash()
current_block=daemon.get_block({'blockhash': bhash})

In [11]:
block_count = current_block['height']
current_block

{u'bits': u'1b00ed47',
 u'chainwork': u'000000000000000000000000000000000000000000000000548fcf6f89072373',
 u'confirmations': 2,
 u'difficulty': 70706.11856510215,
 u'hash': u'59a55f01ba48eaf0e804ad1bce2ad5d45a318ec824319851deaf1592b82c5102',
 u'height': 37280,
 u'mediantime': 1472603204,
 u'merkleroot': u'4bc169bd1625fc324b469f5b19c1070a51a27f67531311022b231718bb6b28fe',
 u'nameclaimroot': u'012b9450aa8142d9dfce517f9ff67375cad688981a12a29b37cec635ad3d4015',
 u'nextblockhash': u'9b821adbbd4da9a9b2a927e09b2061b45ef469c3d3dc1123ca5fc5d122e04d38',
 u'nonce': 2681691226,
 u'previousblockhash': u'6acb363b2f0a4d4ddd9fe5d5d28942484ca6dc2d0f8ad22f323de184e6fae0a2',
 u'size': 227,
 u'target': u'000000000000ed47000000000000000000000000000000000000000000000000',
 u'time': 1472603710,
 u'tx': [u'4bc169bd1625fc324b469f5b19c1070a51a27f67531311022b231718bb6b28fe'],
 u'version': 536870912,
 u'versionHex': u'20000000'}

In [ ]:
vel_fail = []
for j in range(block_count,0,-1):
    try:
        velocity = block_velocity(current_block)
    except:
        velocity = None
        print "block "+str(j)+" velocity computation failed "
        vel_fail = vel_fail+[j]
        
    block_data[j] = {"height":j, 
                     "timestamp":datetime.datetime.fromtimestamp(int(current_block['time'])), 
                     "reward":reward(j)/COIN, 
                     "tx_count": len(current_block['tx']),
                     "tx_outs": velocity,
                     "difficulty": current_block['difficulty'],
                     "confirmations": current_block['confirmations']
                    }
    del velocity
    previous_blockhash = current_block['previousblockhash']
    current_block=daemon.get_block({'blockhash': previous_blockhash})


block 37234 velocity computation failed 


In [ ]:
df = pd.DataFrame(block_data).T.order_by("height")

In [ ]:
df.head()

In [ ]:
df['mined_supply'] = df[df.height>0].reward.cumsum()
df['total_supply'] = df.mined_supply+premine

In [ ]:
df.to_csv('lbry_history.csv')